# Task 4.9 Part 1 Data Prep and Merge

## Contents
## 1. Data Wrangling
## 2. Data quality and consistency checks
## 3. Combine customer data with prepared Instacart data
## 4. Using the merge function, because 2 df's should be combined on the same key in a wide format 

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Importing new data set
path = r'/Users/sophie/Desktop/CareerFoundry /09 2023 Phython'
path

'/Users/sophie/Desktop/CareerFoundry /09 2023 Phython'

In [28]:
customer_ds = pd.read_csv(os.path.join(path, 'Data', 'original data ', 'customers.csv'))
customer_ds.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [29]:
customer_ds.shape

(206209, 10)

In [30]:
# Check values
customer_ds[['date_joined', 'n_dependants']].value_counts()

# There is variety, looks informative

date_joined  n_dependants
4/10/2018    2               71
3/26/2018    3               70
1/8/2020     3               68
6/2/2019     1               66
9/21/2019    2               66
                             ..
4/29/2017    1               25
6/22/2017    0               25
8/6/2018     3               24
3/27/2017    0               24
5/29/2017    3               23
Name: count, Length: 4748, dtype: int64

## 1. Data Wrangling

In [31]:
# Dropping columns
# customer_df = customer_ds(columns = [''])

# Any missings?
#missing_in_column= customer_ds['column'].isna().any()
missing_counts = customer_ds.isna().sum()
print(missing_counts)

# First Name has many missings, might be obsolete. As I cannot know which columns I might need later, I do not delete anything by now.

user_id             0
First Name      11259
Surnam              0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64


In [32]:
# Renaming columns

# Define a list of new column names in the desired order
new_column_names = ['user_id',  'first_name', 'surname', 'gender', 'state', 'age', 'date_joined', 'n_dependants', 'fam_status', 'income']

# Create a copy to keep original data set names
new_df = customer_ds.copy()

# Assign the new column names to the DataFrame's .columns attribute
new_df.columns = new_column_names 

# Print the DataFrame with renamed columns
print(new_df)

        user_id first_name   surname  gender           state  age date_joined  \
0         26711    Deborah  Esquivel  Female        Missouri   48    1/1/2017   
1         33890   Patricia      Hart  Female      New Mexico   36    1/1/2017   
2         65803    Kenneth    Farley    Male           Idaho   35    1/1/2017   
3        125935   Michelle     Hicks  Female            Iowa   40    1/1/2017   
4        130797        Ann   Gilmore  Female        Maryland   26    1/1/2017   
...         ...        ...       ...     ...             ...  ...         ...   
206204   168073       Lisa      Case  Female  North Carolina   44    4/1/2020   
206205    49635     Jeremy   Robbins    Male          Hawaii   62    4/1/2020   
206206   135902      Doris  Richmond  Female        Missouri   66    4/1/2020   
206207    81095       Rose   Rollins  Female      California   27    4/1/2020   
206208    80148    Cynthia     Noble  Female        New York   55    4/1/2020   

        n_dependants fam_st

In [38]:
# Checking data types
new_df.dtypes

user_id          int64
first_name      object
surname         object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [41]:
# Changing data types were necessary

new_df['user_id'] = new_df['user_id'].astype('str')
#new_df['date_joined'] = new_df['date_joined'].astype('int64') # does not work because of the / in date format

In [42]:
# Checking summary stats
new_df.describe()

,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [ ]:
# Looks all reasonable

## 2. Data quality and consistency checks
#### Check for and address missing values and duplicates, and convert any mixed-type data

In [43]:
# Checking whether data frame contains any mixed-type columns
for col in new_df.columns.tolist():
  weird = (new_df[[col]].applymap(type) != new_df[[col]].iloc[0].apply(type)).any(axis = 1) # Test that checks whether the data types within the column are consistent. 
  if len (new_df[weird]) > 0: 
    print (col)
    
# Here, the if statement is checking whether weird is true or false (boolean). If it’s greater than 0 (boolean), than it’s true. If not, it’s false. 
# If weird is true, the command print(col) is executed, which prints the problematic column for you to see.

first_name


In [46]:
# Handle mixed data types

new_df['first_name'] = new_df['first_name'].astype('str')

# Check the data type
column_data_type = new_df['first_name'].dtype

# Print the data type
print(column_data_type)

object


In [47]:
# Missing values

# Find missing values
new_df.isnull().sum()

user_id         0
first_name      0
surname         0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [51]:
missing_counts = new_df.isna().sum()
print(missing_counts)

user_id         0
first_name      0
surname         0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64


#### After changing data type of first_name to consistent object format, there are no missings anymore.

In [52]:
# Check if the value 999 exists in any column
value_exists = (new_df == 999).any().any()

# Print the result
print(value_exists)


False


#### No other indications of missings.

In [53]:
new_df.shape

(206209, 10)

In [55]:
# Check Duplicates

# Look for full duplicates
df_dups = new_df[new_df.duplicated()]
df_dups

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income


In [ ]:
# No duplicates

##  3. Combine customer data with prepared Instacart data

In [57]:
# Import dataset

ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'prepared data ', 'ords_prods_merge_price_clean.pkl'))
ords_prods_merge.shape

(32404859, 26)

In [58]:
pd.options.display.max_columns = None
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours,busiest_period_of_day,max_order,loyalty_flag,order_regularity,order regularity flag,average_spend,spending flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,Most orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,Most orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender


In [59]:
# Check if 'prices' cleaning was successful
ords_prods_merge['prices'].max()

25.0

#### Key columns: user_id

In [62]:
# Check type
# ords_prods_merge.dtypes OR

ords_prods_merge['user_id'].dtype

dtype('int64')

In [64]:
# Adapt to object like in customer df (although int would safe memory, but then it would be included in calculations)
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')
ords_prods_merge['user_id'].dtype

dtype('O')

## 4. Using the merge function, because 2 df's should be combined on the same key in a wide format 

In [66]:
# Excluding _merge from combined data set, because we want to check merge flag in new combined data set 
ords_prods_merge_del_merge = ords_prods_merge.drop(columns = ['_merge'])
ords_prods_merge_del_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_hours,busiest_period_of_day,max_order,loyalty_flag,order_regularity,order regularity flag,average_spend,spending flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,Most orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,Most orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender


In [67]:
ords_prods_merge_del_merge.shape

(32404859, 25)

In [68]:
# Merging data sets, checking with merge flag
# Indicator is user_id
# We want the default: inner join, because otherwise, we have uninformative missings if there are user_ids in each data frame that do not match.
df_merge_instacard_cust = ords_prods_merge_del_merge.merge(new_df, on = 'user_id', indicator = True)
pd.options.display.max_columns = None
df_merge_instacard_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_hours,busiest_period_of_day,max_order,loyalty_flag,order_regularity,order regularity flag,average_spend,spending flag,first_name,surname,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,Most orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,Average orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,Most orders,10,New customer,20.5,Non-frequent customer,6.367797,Low spender,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [69]:
df_merge_instacard_cust.shape

(32404859, 35)

In [70]:
# Confirm results using merge flag
df_merge_instacard_cust['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [ ]:
# Looks valid, only full matches, double check with type=outer

In [72]:
# Double check with type=outer

test_df = ords_prods_merge_del_merge.merge(new_df, on = 'user_id', how = 'outer', indicator = True)

test_df['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [73]:
# Export data to pkl
df_merge_instacard_cust.to_pickle(os.path.join(path, 'Data', 'prepared data ', 'df_merge_instacard_cust.pkl'))